## Nettoyage du DataFrame

Le but de ce Notebook est de nettoyer notre DataFrame (valeurs manquantes, clients inconnus, doublons, valeurs abérantes) afin de pouvoir ensuite faire une analyse approfondie pertinente.

## Décisions de nettoyage

- Suppression des lignes sans Customer ID
- Suppression des doublons
- Exclusion des quantités et prix négatifs
- Conversion des dates au format datetime

Import des packages nécessaires et vérification des valeurs manquantes.

In [68]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/online_retail.csv')
df.head()
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

On observe que la colonne Description possède 4382 valeurs manquantes et la colonne Customer ID en a 243007

Les valeurs manquantes sur Customer ID sont cohérentes avec le contexte métier et correspondent à des clients non identifiés.

Renommage de la colonnes Customer ID en supprimant l'espace

In [69]:
df = df.rename(columns={
    'Customer ID': 'CustomerID'
})

Vérifions le nombre de lignes et colonnes de notre DataFrame

In [70]:
df.shape

(1067371, 8)

Notre DataFrame contient 1067371 Lignes et 8 colonnes

In [71]:
df = df.dropna(subset=['CustomerID'])

Les lignes sans identifiants client ont été supprimées car elles ne permettent pas de réaliser des analyses clients pertinentes

Vérifions de nouveau le nombre de lignes dans notre DataFrame

In [72]:
df.shape

(824364, 8)

 Notre DataFrame contient à présent 824364 lignes, on voit bien que nous avons supprimé un certain nombres de lignes.

Identification et suppression des doublons

In [73]:
df.duplicated().sum()
df = df.drop_duplicates()


Vérifions encore une fois le nombre de lignes dans notre DataFrame

In [74]:
df.shape

(797885, 8)

Il reste maintenant 797885 lignes après suppression des doublons.

On vérifie les types de chaque colonnes

In [75]:
df.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
CustomerID     float64
Country         object
dtype: object

On convertit la date en type datetime64

In [76]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

On vérifie que cela a bien fonctionné

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 797885 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      797885 non-null  object        
 1   StockCode    797885 non-null  object        
 2   Description  797885 non-null  object        
 3   Quantity     797885 non-null  int64         
 4   InvoiceDate  797885 non-null  datetime64[ns]
 5   Price        797885 non-null  float64       
 6   CustomerID   797885 non-null  float64       
 7   Country      797885 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 54.8+ MB


Ici on identifie les lignes qui ont une quantité négative ou nulle ce qui ne serait pas pertinent pour notre analyse. Retour produit ? Commande annulée ou supprimée ?

In [78]:
df[df['Quantity'] <= 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [79]:
df = df[df['Quantity'] > 0]

Dans la même idée ici nous identifions les prix négatifs ou nuls puis nous les retirons de notre DataFrame

In [80]:
df[df['Price'] <= 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
4674,489825,22076,6 RIBBONS EMPIRE,12,2009-12-02 13:34:00,0.0,16126.0,United Kingdom
6781,489998,48185,DOOR MAT FAIRY CAKE,2,2009-12-03 11:19:00,0.0,15658.0,United Kingdom
16107,490727,M,Manual,1,2009-12-07 16:38:00,0.0,17231.0,United Kingdom
18738,490961,22065,CHRISTMAS PUDDING TRINKET POT,1,2009-12-08 15:25:00,0.0,14108.0,United Kingdom
18739,490961,22142,CHRISTMAS CRAFT WHITE FAIRY,12,2009-12-08 15:25:00,0.0,14108.0,United Kingdom
...,...,...,...,...,...,...,...,...
1004540,577129,22464,HANGING METAL HEART LANTERN,4,2011-11-17 19:52:00,0.0,15602.0,United Kingdom
1005014,577168,M,Manual,1,2011-11-18 10:42:00,0.0,12603.0,Germany
1006110,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,2011-11-18 13:23:00,0.0,12444.0,Norway
1011446,577696,M,Manual,1,2011-11-21 11:57:00,0.0,16406.0,United Kingdom


In [81]:
df = df[df['Price'] > 0]

Vérifions à nouveau le nombre de lignes

In [82]:
df.shape

(779425, 8)

Il reste à présent 779425 lignes dans notre DataFrame

Nous créons une nouvelle colonne TotalPrice qui sera le résultat de la multiplication de la colonne Quantity et la colonne Price afin de calculer le chiffre d'affaires

In [83]:
df['TotalPrice'] = df['Quantity'] * df['Price']

Sauvegarde du dataset nettoyé.
Le dataset nettoyé servira de base pour les analyses de ventes et de comportement client.

In [84]:
df.to_csv('../data/online_retail_clean.csv', index=False)

## Conclusion du nettoyage

Les données ont été nettoyées en supprimant les valeurs manquantes critiques, les doublons et les lignes incohérentes comme les prix et quantité négatifs ou nuls.
Une variable de chiffre d'affaires a été ajoutée afin de faciliter les analyses futures.